In [2]:
import matplotlib.pyplot as plt
import numpy as np

In [15]:
def find_ends_supergauss(line_data, gauss_length=10, threshold_Imax=0.5, plotting=True):
    xdata = np.arange(len(line_data))
    line_data_temp = line_data
    line_data = np.array([float(value)/max(line_data_temp) for value in line_data_temp])
    initial_guess = [.2,1.,np.median(xdata),gauss_length, 6]
    constraints =([0, 0, 0, gauss_length, 0.],[np.inf, np.inf, np.inf, np.inf, np.inf])
    popt, pcov = curve_fit(super_gauss_function, xdata, line_data,
                           p0 = initial_guess, bounds = constraints)
    fine_scale_x = np.linspace(xdata[0],xdata[-1],len(xdata)*1000)
    maximum = max(super_gauss_function( xdata, *popt))
    #--- this is the threshold
    intersection_value = maximum-(maximum - popt[0])*threshold_Imax
    linedata = [intersection_value for x in fine_scale_x]
    index = np.argwhere(np.diff(np.sign(linedata - super_gauss_function(fine_scale_x, *popt)))).flatten()
    index_intersection = [fine_scale_x[index[0]], fine_scale_x[index[1]]]
    # index_intersection = [int(fine_scale_x[index[0]]), int(fine_scale_x[index[1]])]
    #-- Here is where we ensure two crossings and select for length
    if len(index)==2 and plotting: 
        plt.plot(xdata, line_data)
        plt.plot(fine_scale_x, super_gauss_function(fine_scale_x, *popt), 'g--')
        plt.plot(index_intersection,
                super_gauss_function([fine_scale_x[index[0]],fine_scale_x[index[1]]], *popt), 'ro')
        plt.show()
    return index_intersection


def super_gauss_function(x, floor, amplitude, mean, sigma, power):
    '''https://en.wikipedia.org/wiki/Gaussian_function'''
    return floor + amplitude*np.exp((-((x-mean)**2/(2*sigma**2))**power))
def plotplay_supergauss(floor=1, amplitude=1, mean=10, sigma=2, power=5):
    fig, ax = plt.subplots()
    x = np.linspace(mean - 3 * sigma, mean + 3 * sigma, 100)
    y_gauss = super_gauss_function(x, floor, amplitude, mean, sigma, power)
    ax.plot(x, y_gauss)
    ax.set_xlabel('pixels')
    ax.set_label('Intensity')
    ax.set_title('Super Gaussian')
    ax.grid(True)

In [18]:
from ipywidgets import interact, interactive
interact(plotplay_supergauss, floor=(0, 10), amplitude=(0, 10), mean=(1, 100), sigma=(1, 10), power=(0, 100),
         continuous_update=False);

interactive(children=(IntSlider(value=1, description='floor', max=10), IntSlider(value=1, description='amplitu…